# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# Initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# For every filepath in the file path list 
for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # Creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# Uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# Creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [3]:
# Make a connection to a Cassandra instance your local machine 
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [4]:
# Create a Keyspace called "udacity"
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [5]:
# Set KEYSPACE to the keyspace "udacity"
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1
In query 1, we want to get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.
The table songs_per_session will be created for this aim. The columns are:
- sessionId (int)
- itemInSession (int)
- artist (text)
- song (text)
- length (float)

The column (sessionId) was used as a partition key because the queries will filter by this column. Columns (itemInSession, song) were used as clustering columns to guarantee the uniqueness of the PRIMARY KEY.

In [17]:
query = "CREATE TABLE IF NOT EXISTS songs_per_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession, song))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [18]:
# Save the data from event_datafile_new.csv into the right columns of songs_per_session
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_per_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        # Assign column elements (sessionId, itemInSession, artist, song, length) in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# Execute Query 1:
query = "SELECT artist, song, length FROM songs_per_session WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### The output should be:
> Faithless Music Matters (Mark Knight Dub) 495.30731201171875

#### Query 2
In query 2, we want to get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.
The table user_songs_per_session will be created for this aim. The columns are:
- userId (int) 
- sessionId (int)
- itemInSession (int)
- artist (text)
- song (text)
- firstName (text)
- secondName (text)

The columns (userId, sessionId) were used as a partition key because the queries will filter by these columns. Column "itemInSession" was used as clustering column to sort the table by this item.

In [19]:
query = "CREATE TABLE IF NOT EXISTS user_songs_per_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, secondName text, \
            PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Save the data from event_datafile_new.csv into the right columns of user_songs_per_session
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs_per_session (userId, sessionId, itemInSession, artist, song, firstName, secondName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign column elements (userId, sessionId, itemInSession, artist, song, firstName, secondName) in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

# Execute Query 2:
query = "SELECT artist, song, firstname, secondname, iteminsession FROM user_songs_per_session WHERE userId=10 AND sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    # itemInSession is added in the output to prove that it works correcting (ASC order)
    print (row.artist, row.song, row.firstname, row.secondname, row.iteminsession)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


### The output should be:
> Down To The Bone Keep On Keepin' On Sylvie Cruz 0

> Three Drives Greece 2000 Sylvie Cruz 1

> Sebastien Tellier Kilometer Sylvie Cruz 2

> Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3

#### Query 3
In query 3, we want to get every user name (first and last) who listened to the song "All Hands Against His Own".
The table users_per_song will be created for this aim. The columns are:
- song (text)
- userId (int)
- firstName (text)
- lastName (text)

The column "song" was used as a partition key because the queries will filter by this column. Column "userId" was used as clustering column to help make up a unique key.

In [20]:
query = "CREATE TABLE IF NOT EXISTS users_per_song"
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

# Save the data from event_datafile_new.csv into the right columns of users_per_song
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_per_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign columns (song, userId, firstName, lastName) in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

# Execute Query 3:
query = "SELECT firstname, lastname FROM users_per_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)              

Jacqueline Lynch
Tegan Levine
Sara Johnson


### The output should be:
> Jacqueline Lynch

> Sara Johnson

> Tegan Levine

### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP table IF EXISTS songs_per_session"
query1 = "DROP table IF EXISTS users_per_song"
query2 = "DROP table IF EXISTS user_songs_per_session"
session.execute(query)
session.execute(query1)
session.execute(query2)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()